In [2]:
import bayes3d as b
import numpy as np 
import jax.numpy as jnp
import matplotlib.pyplot as plt
import jax
import time
import PIL.Image
from math import sqrt
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML

/opt/conda/envs/bayes3d/lib/python3.9/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


In [3]:
def display_video(frames, framerate=30):
    if type(frames[0]) == PIL.Image.Image:
      frames = [np.array(frames[i]) for i in range(len(depths))]
    height, width, _ = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())

In [4]:
intrinsics = b.Intrinsics(
    height=360,
    width=480,
    fx=180*sqrt(3), fy=180*sqrt(3),
    cx=240.0, cy=180.0,
    near=0.1, far=100.0
)
# intrinsics = b.Intrinsics(
#     height=300,
#     width=300,
#     fx=200.0, fy=200.0,
#     cx=150.0, cy=150.0,
#     near=0.001, far=6.0
# )

In [5]:
b.setup_renderer(intrinsics)
# b.RENDERER.add_mesh_from_file("assets/cube.obj",mesh_name="cube_0", scaling_factor=0.5)

[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (480, 384, 1024)


In [6]:
b.RENDERER.add_mesh_from_file("assets/cube.obj",mesh_name="cube_0", scaling_factor=[0.5,0.5,0.5])
b.RENDERER.add_mesh_from_file("assets/plane.obj",mesh_name="occluder", scaling_factor=[0.0667,0.0667,0.0667])

In [7]:
loaded_poses = np.load('poses_camera_frame.npz')

In [8]:
occ_pose = loaded_poses['occ']
occ_pose[1:3] *= -1
occ_pose = jnp.array(occ_pose)

loaded_box_poses = loaded_poses['box']
gt_poses = []
occ_poses = []
for i in range(loaded_box_poses.shape[0]):
    pose = loaded_box_poses[i,...]
    pose[1:3] *= -1 # convert to CV2 convention (bayes3d currently assumes input is in cv2 coordinate frame)
    gt_poses.append(jnp.array(pose))
    occ_poses.append(occ_pose)
gt_poses = jnp.stack(gt_poses)
occ_poses = jnp.stack(occ_poses)

total_gt_poses = jnp.stack([gt_poses, occ_poses])

In [9]:
# gt_images = jax.vmap(b.RENDERER.render_single_object, in_axes=(0, None))(gt_poses, jnp.int32(0))
gt_images = b.RENDERER.render_multiobject_parallel(total_gt_poses, [0,1])

# gt_images = jnp.stack([b.RENDERER.render_multiobject(
#     jnp.array([gt_poses[i], occ_pose]),
#     [0, 1]
# ) for i in range(gt_poses.shape[0])])



In [11]:
depths = [b.viz.get_depth_image(gt_images[i,:,:,2]) for i in range(gt_images.shape[0])]
# b.make_gif(depths, "test.gif")

In [12]:
display_video(depths)

In [13]:
translation_deltas = b.make_translation_grid_enumeration(-0.2, -0.2, -0.2, 0.2, 0.2, 0.2, 5, 5, 5)


dx, dy, dz = 2, 2, 2
gridding = [
    b.make_translation_grid_enumeration(
        -dx, -dy, -dz, dx, dy, dz, 5,5,5
    ),
    b.make_translation_grid_enumeration(
        -dx/2.0, -dy/2, -dz/2, dx/2, dy/2, dz/2, 5,5,5
    ),
    b.make_translation_grid_enumeration(
        -dx/10.0, -dy/10, -dz/10, dx/10, dy/10, dz/10, 5,5,5
    )
]



key = jax.random.PRNGKey(3)
rotation_deltas = jax.vmap(lambda key: b.distributions.gaussian_vmf(key, 0.00001, 800.0))(
    jax.random.split(key, 100)
)

In [14]:
pose_estimate = gt_poses[0]
occ_poses_trans = jnp.stack([occ_pose for _ in range(translation_deltas.shape[0])])
occ_poses_rot = jnp.stack([occ_pose for _ in range(rotation_deltas.shape[0])])

In [15]:
def update_pose_estimate(pose_estimate, gt_image):
    proposals = jnp.einsum("ij,ajk->aik", pose_estimate, translation_deltas)
    rendered_images = b.RENDERER.render_multiobject_parallel(jnp.stack([proposals, occ_poses_trans]), jnp.array([0,1]))
    weights_new = b.threedp3_likelihood_parallel(gt_image, rendered_images, 0.05, 0.1, 10**3, 3)
    pose_estimate = proposals[jnp.argmax(weights_new)]

    proposals = jnp.einsum("ij,ajk->aik", pose_estimate, rotation_deltas)
    rendered_images = b.RENDERER.render_multiobject_parallel(jnp.stack([proposals, occ_poses_rot]), jnp.array([0,1]))
    weights_new = b.threedp3_likelihood_parallel(gt_image, rendered_images, 0.05, 0.1, 10**3, 3)
    pose_estimate = proposals[jnp.argmax(weights_new)]
    return pose_estimate, pose_estimate

def update_pose_estimate_c2f(pose_estimate, gt_image):
    for translation_deltas in gridding:
        proposals = jnp.einsum("ij,ajk->aik", pose_estimate, translation_deltas)
        rendered_images = b.RENDERER.render_multiobject_parallel(jnp.stack([proposals, occ_poses_trans]), jnp.array([0,1]))
        weights_new = b.threedp3_likelihood_parallel(gt_image, rendered_images, 0.05, 0.1, 10**3, 3)
        pose_estimate = proposals[jnp.argmax(weights_new)]

    proposals = jnp.einsum("ij,ajk->aik", pose_estimate, rotation_deltas)
    rendered_images = b.RENDERER.render_multiobject_parallel(jnp.stack([proposals, occ_poses_rot]), jnp.array([0,1]))
    weights_new = b.threedp3_likelihood_parallel(gt_image, rendered_images, 0.05, 0.1, 10**3, 3)
    pose_estimate = proposals[jnp.argmax(weights_new)]
    return pose_estimate, pose_estimate

In [16]:
# inference_program = jax.jit(lambda p,x: jax.lax.scan(update_pose_estimate, p,x)[1])
inference_program = jax.jit(lambda p,x: jax.lax.scan(update_pose_estimate_c2f, p,x)[1])

In [17]:
start = time.time()
pose_estimates_over_time = inference_program(gt_poses[0], gt_images)
end = time.time()
print ("Time elapsed:", end - start)
print ("FPS:", gt_poses.shape[0] / (end - start))

Time elapsed: 14.212560415267944
FPS: 6.332426907632833


In [18]:
viz_images = []
max_depth = 10.0

pose_estimates = jnp.stack([pose_estimates_over_time, occ_poses])

rerendered_images = b.RENDERER.render_multiobject_parallel(pose_estimates_over_time[None, ...], jnp.array([0]))
viz_images = [
    b.viz.multi_panel(
        [b.viz.get_depth_image(d[:,:,2]), b.viz.get_depth_image(r[:,:,2])],
        ["Ground Truth", "Inferred Reconstruction"],
    )
    for (r, d) in zip(rerendered_images, gt_images)
]

In [19]:
display_video(viz_images)

In [21]:
b.make_gif(viz_images, "/home/ubuntu/.scp/occluder_failure_c2f.gif")